# Naive Bayes - probability classifier

    Example that builds a probability classifier of the chance that a message contains a specific word and therefore is classified as 'spam'.

In [47]:
from __future__ import division
from collections import Counter, defaultdict
from machine_learning import split_data
import math, random, re, glob

In [48]:
# prepare data from a text file
# tokenize messages into distinct words

def tokenize(message):
    message = message.lower()                       # convert to lowercase
    all_words = re.findall("[a-z0-9']+", message)   # extract the words
    return set(all_words)                           # remove duplicates

In [49]:
# count how many times a word appears in a spam message and a non spam message

def count_words(training_set):
    """training set consists of pairs (message, is_spam)"""
    counts = defaultdict(lambda: [0, 0])
    for message, is_spam in training_set:
        for word in tokenize(message):
            counts[word][0 if is_spam else 1] += 1
    return counts

In [38]:
# turn these counts into estimated probabilities for each word 
# and the probability of seeing that word in a spam message and in a non spam message

def word_probabilities(counts, total_spams, total_non_spams, k=0.5):
    """turn the word_counts into a list of triplets 
    w, p(w | spam) and p(w | ~spam)"""
    return [(w,
             (spam + k) / (total_spams + 2 * k),
             (non_spam + k) / (total_non_spams + 2 * k))
             for w, (spam, non_spam) in counts.iteritems()]

In [39]:
# assign probabilities to messages

def spam_probability(word_probs, message):
    message_words = tokenize(message)
    log_prob_if_spam = log_prob_if_not_spam = 0.0

    for word, prob_if_spam, prob_if_not_spam in word_probs:

        # for each word in the message, 
        # add the log probability of seeing it 
        if word in message_words:
            log_prob_if_spam += math.log(prob_if_spam)
            log_prob_if_not_spam += math.log(prob_if_not_spam)

        # for each word that's not in the message
        # add the log probability of _not_ seeing it
        else:
            log_prob_if_spam += math.log(1.0 - prob_if_spam)
            log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)
            
    prob_if_spam = math.exp(log_prob_if_spam)
    prob_if_not_spam = math.exp(log_prob_if_not_spam)
    return prob_if_spam / (prob_if_spam + prob_if_not_spam)

In [40]:
# all together this gives:
class NaiveBayesClassifier:

    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = []

    def train(self, training_set):
    
        # count spam and non-spam messages
        num_spams = len([is_spam 
                         for message, is_spam in training_set 
                         if is_spam])
        num_non_spams = len(training_set) - num_spams

        # run training data through our "pipeline"
        word_counts = count_words(training_set)
        self.word_probs = word_probabilities(word_counts, 
                                             num_spams, 
                                             num_non_spams,
                                             self.k)
                                             
    def classify(self, message):
        return spam_probability(self.word_probs, message)

# Implementation

In [41]:
import glob, re

In [42]:
# modify the path with wherever you've put the files
path = r"/Users/annalie/Dev/data-science-from-scratch/data/spam-mails/*/*"
data = []

In [43]:
# investigate the mails to be spam or not by only looking at the subject lines of each mail

# glob.glob returns every filename that matches the wildcarded path
for fn in glob.glob(path):
    is_spam = "ham" not in fn
    
    with open(fn,'r') as file: 
        for line in file:
            if line.startswith("Subject:"):
                # remove the leading "Subject: " and keep what's left 
                subject = re.sub(r"^Subject: ", "", line).strip()
                data.append((subject, is_spam))

In [51]:
# split the data into training and test set and build the classifier

random.seed(0) 
train_data, test_data = split_data(data, 0.75)

classifier = NaiveBayesClassifier()
classifier.train(train_data)

In [55]:
# check how well the model does

classified = [(subject, is_spam, classifier.classify(subject))
             for subject, is_spam in test_data]

# > 0.5 is spam; < 0.5 is non spam
# count the combinations of actual is_spam and predicted is_spam
counts = Counter((is_spam, spam_probability > 0.5)
                 for _, is_spam, spam_probability in classified)

print "actual is_spam vs predicted is_spam: "
counts

actual is_spam vs predicted is_spam: 


Counter({(False, False): 704,
         (False, True): 33,
         (True, False): 38,
         (True, True): 101})

    "true positives (spam classified as 'spam'): 101"
    "false positives (ham classified as 'spam'): 33"
    "true negatives (ham classified as 'ham'): 704"
    "false negatives (spam classified as 'ham'): 38"

In [57]:
print "precision = ", 101/(101 + 33)
print "recall = ", 101/ (101 + 38)

precision =  0.753731343284
recall =  0.726618705036


In [61]:
# look at the most misclassified mails

# sort by spam_probability from smallest to largest
classified.sort(key = lambda row: row[2])

# the highest predicted spam probabilities among the non-spam
most_spam_hams = filter(lambda row: not row[1], classified)[:5]
print most_spam_hams

print "---"
# the lowest predicted spam probabilities among the actual spams
most_ham_spams = filter(lambda row: row[1], classified)[:5]
print most_ham_spams

[('Re[2]: Selling Wedded Bliss (was Re: Ouch...)', False, 3.048747404866875e-10), ('Re[2]: Selling Wedded Bliss (was Re: Ouch...)', False, 3.048747404866875e-10), ('Re[2]: Selling Wedded Bliss (was Re: Ouch...)', False, 3.048747404866875e-10), ('Re[2]: Selling Wedded Bliss (was Re: Ouch...)', False, 3.048747404866875e-10), ('Re: [Razor-users] Problem with Razor 2.14 and Spamassassin 2.41', False, 6.20932440680424e-10)]
---
[('Re: girls', True, 0.0009525186158414711), ('Introducing Chase Platinum for Students with a 0% Introductory APR', True, 0.0012566691211091526), ('.Message report from your contact page....//ytu855 rkq', True, 0.0015109358288617285), ('Testing a system, please delete', True, 0.0026920538836874555), ('Never pay for the goodz again (8SimUgQ)', True, 0.00591162322193142)]


In [69]:
# find the most common words for spam messages
def p_spam_given_word(word_prob):
    """uses bayes's theorem to comput p(spam | message contains word)"""
    
    # word_prob is one o the triplets produed by word_probabilities
    word, prob_if_spam, prob_if_not_spam = word_prob
    return prob_if_spam / (prob_if_spam + prob_if_not_spam)

words = sorted(classifier.word_probs, key = p_spam_given_word)
words[:5]

[('spambayes', 0.0013698630136986301, 0.04601648351648352),
 ('users', 0.0013698630136986301, 0.036401098901098904),
 ('razor', 0.0013698630136986301, 0.030906593406593408),
 ('zzzzteana', 0.0013698630136986301, 0.029075091575091576),
 ('sadev', 0.0013698630136986301, 0.026785714285714284)]

In [68]:
most_spam_words = words[-5:]
most_spam_words

[('year', 0.028767123287671233, 0.00022893772893772894),
 ('sale', 0.031506849315068496, 0.00022893772893772894),
 ('rates', 0.031506849315068496, 0.00022893772893772894),
 ('systemworks', 0.036986301369863014, 0.00022893772893772894),
 ('money', 0.03972602739726028, 0.00022893772893772894)]

In [67]:
most_ham_words = words[:5]
most_ham_words

[('spambayes', 0.0013698630136986301, 0.04601648351648352),
 ('users', 0.0013698630136986301, 0.036401098901098904),
 ('razor', 0.0013698630136986301, 0.030906593406593408),
 ('zzzzteana', 0.0013698630136986301, 0.029075091575091576),
 ('sadev', 0.0013698630136986301, 0.026785714285714284)]

# Example from Machine Learning Mastery
http://machinelearningmastery.com/naive-bayes-classifier-scratch-python/

In [70]:
# Example of Naive Bayes implemented from Scratch in Python
import csv
import random
import math

def loadCsv(filename):
    lines = csv.reader(open(filename, "rb"))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset

def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

def mean(numbers):
    return sum(numbers)/float(len(numbers))

def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for classValue, instances in separated.iteritems():
        summaries[classValue] = summarize(instances)
    return summaries

def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.iteritems():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities

def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.iteritems():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

def getAccuracy(testSet, predictions):
    correct = 0
    for i in range(len(testSet)):
        if testSet[i][-1] == predictions[i]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

def main():
    filename = '/Users/annalie/Dev/data-science-from-scratch/data/pima-indians-diabetes-data.csv'
    splitRatio = 0.67
    dataset = loadCsv(filename)
    trainingSet, testSet = splitDataset(dataset, splitRatio)
    print('Split {0} rows into train={1} and test={2} rows').format(len(dataset), len(trainingSet), len(testSet))
    # prepare model
    summaries = summarizeByClass(trainingSet)
    # test model
    predictions = getPredictions(summaries, testSet)
    accuracy = getAccuracy(testSet, predictions)
    print('Accuracy: {0}%').format(accuracy)

main()

Split 768 rows into train=514 and test=254 rows
Accuracy: 74.4094488189%
